<a href="https://colab.research.google.com/github/stevegbrooks/big-portfolio-learner/blob/big-data/step2_big_portfolio_learner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIS 545 Final Project

## Big Portfolio Learner: Time Series Analysis

### Team members: Steven Brooks & Chenlia Xu

In [ ]:
import random
import numpy as np 
import json
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import datetime
import glob
import seaborn as sns
import re
import os

In [ ]:
%%capture
## If statsmodels not the latest version uncomment the following:
!pip install statsmodels --upgrade
## If boto3 not already installed uncomment the following:
!pip3 install boto3

In [ ]:
import boto3
from botocore import UNSIGNED

from botocore.config import Config

s3 = boto3.resource('s3', config=Config(signature_version=UNSIGNED))
s3.Bucket('cis545project').download_file('data/stock_data_2002_2019.csv', 'stock_data_2002_2019.csv')
s3.Bucket('cis545project').download_file('data/technical_data.zip', 'technical_data.zip')

In [ ]:
%%capture

tech_dir = "technical_data"
if not os.path.exists(tech_dir):
  os.makedirs(tech_dir)
!unzip /content/technical_data.zip -d /content/$tech_dir
!rm -f $tech_dir/.gitempty

# Setup Spark Session

In [ ]:
%%capture

!apt install libkrb5-dev
!pip install findspark
!pip install sparkmagic
!pip install pyspark
!pip install pyspark --user
!pip install seaborn --user
!pip install imageio --user
!pip install folium --user

!apt update
!apt install gcc python-dev libkrb5-dev

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

import os

spark = SparkSession.builder.getOrCreate()

In [ ]:
%load_ext sparkmagic.magics

In [ ]:
#graph section
import networkx as nx
# SQLite RDBMS
import sqlite3
# Parallel processing
# import swifter
import pandas as pd
# NoSQL DB
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError, OperationFailure

import os
os.environ['SPARK_HOME'] = '/content/spark-2.4.5-bin-hadoop2.7'
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
import pyspark
from pyspark.sql import SQLContext

try:
    if(spark == None):
        spark = SparkSession.builder.appName('Initial').getOrCreate()
        sqlContext=SQLContext(spark)
except NameError:
    spark = SparkSession.builder.appName('Initial').getOrCreate()
    sqlContext=SQLContext(spark)

We will load the data into the Spark context here.

In [ ]:
stock_data_sdf = spark.read.load(
    'stock_data_2002_2019.csv', 
    format = 'csv', 
    header = 'true', 
    inferSchema = 'true', 
    sep = ','
)

stock_data_sdf.printSchema()

root
 |-- symbol: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- adjusted_close: double (nullable = true)
 |-- volume: long (nullable = true)



# Section 1: Train Test Split

We will train the data using the years 2002 to 2017. Our validation set will be the year 2018. Our test set will be the year 2019.